In [1]:
!pip install --quiet transformers torch langchain sentence-transformers
!pip install langchain-community
!pip install -U langchain langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.74
    Uninstalling langchain-core-0.3.74:
      Successfully uninstalled langchain-core-0.3.74
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.2 

In [2]:
import json
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from huggingface_hub import login
from google.colab import userdata
from typing import List, Dict, Any
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pprint
from langgraph.graph import StateGraph, END


api_key = userdata.get('api_key')

login(api_key)

In [3]:
model_name = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype="auto")

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0.2
)

gemma_llm = HuggingFacePipeline(pipeline=generator)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-3060399952.py:13: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  gemma_llm = HuggingFacePipeline(pipeline=generator)


In [4]:
parse_query_prompt = PromptTemplate(
    input_variables=["user_query"],
    template="""
You are a language parsing assistant. Read the user query.
Understand where they would like to go, for how many days and what is their budget. What is it that they are interested in doing or seeing there?
Extract a JSON object from the user query based on your understanding of it and output it. The JSON must contain these keys:
- destination
- days
- budget
- interests (list of strings)

Fill missing fields with defaults if not mentioned.
Always output **only JSON**. Do NOT add explanations or anything else.

User query: "{user_query}"
"""
)

In [83]:
import json
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph

# Optional helper to append to lists (not needed for dict keys like query)
from langgraph.graph.message import add_messages

# -----------------------------
# Define the state
# -----------------------------
class TravelState(TypedDict):
    user_query: str
    destination: str
    days: int
    budget: float
    interests: list
    origin: str
    chosen_flight: dict
    chosen_hotel: dict
    ranked_places: list
    budget_remaining: float
    itinerary: dict
    final_output: str
    pdf_file: str

In [84]:
def parse_user_query(user_query: str) -> dict:
    # Generate LLM response
    prompt_text = parse_query_prompt.format(user_query=user_query)
    response = gemma_llm(prompt_text)

    # Extract JSON substring to avoid parsing errors
    try:
        start = response.index("{")
        end = response.rindex("}") + 1
        json_str = response[start:end]
        parsed_dict = json.loads(json_str)
    except:
        print("Warning: Failed to parse JSON. Raw response:")
        print(response)
        parsed_dict = {}

    # If the key is missing or empty/None/0, replace with default
    parsed_dict["interests"] = parsed_dict.get("interests") or ["general sightseeing"]
    parsed_dict["days"] = parsed_dict.get("days") or 3
    parsed_dict["budget"] = parsed_dict.get("budget") or 2000
    parsed_dict["destination"] = parsed_dict.get("destination") or "Tokyo"


    # Add fixed origin
    parsed_dict["origin"] = "San Francisco"
    # Save the full raw query for later use
    #parsed_dict["query"] = user_query


    return parsed_dict

In [85]:
def parse_user_query_node(state: TravelState):
    user_query = state.get("user_query", "")
    parsed_dict = parse_user_query(user_query)
    return parsed_dict

In [86]:
def select_flight(
    parsed_request: Dict[str, Any],
    flights_file: str = "flights.json"
) -> Dict[str, Any]:
    # Load flights JSON
    with open(flights_file, "r", encoding="utf-8") as f:
        flights_data = json.load(f)

    # Filter flights matching origin and destination
    matching_flights = [
        flight for flight in flights_data
        if flight["origin"] == parsed_request["origin"] and
           flight["destination"] == parsed_request["destination"]
    ]

    if not matching_flights:
        print("Warning: No flights found for this route.")
        return {"chosen_flight": None, "budget_remaining": parsed_request.get("budget", 0)}

    # Select cheapest flight
    cheapest_flight = min(matching_flights, key=lambda x: x["price"])

    # Calculate remaining budget
    budget_remaining = parsed_request.get("budget", 0) - cheapest_flight["price"]

    return {
        "chosen_flight": cheapest_flight,
        "budget_remaining": budget_remaining
    }


In [87]:
def select_flight_node(state: TravelState) -> Dict[str, Any]:
    # Read the parsed request from state
    parsed_request = {
        "origin": state.get("origin", ""),
        "destination": state.get("destination", ""),
        "days": state.get("days", 0),
        "budget": state.get("budget", 0),
        "interests": state.get("interests", [])
    }

    flights_file = "flights.json"
    flight_info = select_flight(parsed_request, flights_file)
    return flight_info

In [88]:
def select_hotel(
    parsed_request: Dict[str, Any],
    budget_remaining: float,
    hotels_file: str = "hotels.json"
) -> Dict[str, Any]:
    # Load hotels JSON
    with open(hotels_file, "r", encoding="utf-8") as f:
        hotels_data = json.load(f)

    # Filter hotels in the destination city
    city_hotels = [
        hotel for hotel in hotels_data
        if hotel["city"] == parsed_request["destination"]
    ]

    if not city_hotels:
        print("Warning: No hotels found in this city.")
        return {"chosen_hotel": None, "budget_remaining": budget_remaining}

    # Sort hotels by rating (descending)
    city_hotels.sort(key=lambda x: x.get("rating", 0), reverse=True)

    # Pick best hotel that still keeps 25% budget for sightseeing
    total_budget = parsed_request.get("budget", 0)
    chosen_hotel = None
    for hotel in city_hotels:
        cost_for_days = hotel["price_per_night"] * parsed_request.get("days", 3)
        if cost_for_days <= budget_remaining and (budget_remaining - cost_for_days) <= 0.75 * total_budget:
            chosen_hotel = hotel
            budget_remaining -= cost_for_days
            break

    # If no hotel meets 25% rule, pick the cheapest hotel within budget
    if not chosen_hotel:
        for hotel in sorted(city_hotels, key=lambda x: x["price_per_night"]):
            cost_for_days = hotel["price_per_night"] * parsed_request.get("days", 3)
            if cost_for_days <= budget_remaining:
                chosen_hotel = hotel
                budget_remaining -= cost_for_days
                break

    return {
        "chosen_hotel": chosen_hotel,
        "budget_remaining": budget_remaining
    }

In [89]:
def select_hotel_node(state: TravelState) -> Dict[str, Any]:
    # Read the parsed request from state
    parsed_request = {
        "origin": state.get("origin", ""),
        "destination": state.get("destination", ""),
        "days": state.get("days", 0),
        "budget": state.get("budget", 0),
        "interests": state.get("interests", [])
    }
    budget_remaining = state.get("budget_remaining", 0)

    # Call function
    hotel_info = select_hotel(parsed_request, budget_remaining)

    return hotel_info

In [90]:
# function to convert each place entry into a retrievable Document for RAG
def load_places(file_path: str) -> List[Document]:
    """
    Convert each place entry into a retrievable Document for RAG.
    Full text includes name, categories, description, coordinates, duration, ticket cost, opening hours.
    """
    docs = []
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    for i, entry in enumerate(data):
        text = (
            f"Name: {entry['name']}\n"
            f"City: {entry['city']}\n"
            f"Category: {', '.join(entry.get('category', []))}\n"
            f"Coordinates: {entry['coordinates']}\n"
            f"Duration (minutes): {entry['duration']}\n"
            f"Opening Hours: {entry['opening_hours']}\n"
            f"Ticket Cost: {entry['ticket_cost']}\n"
            f"Description: {entry['description']}"
        )
        docs.append(
            Document(
                page_content=text,
                metadata={
                    "type": "place",
                    "index": i,
                    "name": entry['name'],
                    "city": entry['city'],
                    "coordinates": entry['coordinates'],
                    "category": entry.get('category', []),
                    "duration_mins": entry['duration'],
                    "ticket_cost": entry['ticket_cost'],
                    "opening_hours": entry['opening_hours'],
                    "description": entry['description']
                }
            )
        )
    return docs

In [91]:
def rank_places_by_query_rag(
    parsed_request: Dict[str, Any],
    places_file: str = "places.json"
) -> List[Dict[str, Any]]:
    """
    Node 4: Retrieve and rank places using classic full-text RAG, filtered by destination city.

    Input:
        - parsed_request (dict) from Node 1
        - places_file (str): path to places.json
    Output:
        - ranked_places (list of dicts) ordered by similarity to user's query
    """
    # Load places
    places_docs = load_places(places_file)

    # Filter places by destination city
    destination_city = parsed_request.get("destination")
    places_docs = [doc for doc in places_docs if doc.metadata["city"].lower() == destination_city.lower()]

    if not places_docs:
        print(f"Warning: No places found in city {destination_city}.")
        return []

    # Initialize embedding model
    model = SentenceTransformer('all-MiniLM-L6-v2')

    # Embed each place's full text
    place_texts = [doc.page_content for doc in places_docs]
    place_embeddings = model.encode(place_texts)

    # Embed user's query
    query_text = parsed_request.get("query", "general sightseeing")
    query_embedding = model.encode([query_text])[0]

    # Compute cosine similarity
    similarities = cosine_similarity([query_embedding], place_embeddings)[0]

    # Attach similarity score and sort descending
    ranked = []
    for doc, score in zip(places_docs, similarities):
        ranked.append({
            "place": doc.metadata,
            "similarity": score
        })
    ranked.sort(key=lambda x: x["similarity"], reverse=True)

    return ranked

In [92]:
def rank_places_node(state: TravelState) -> Dict[str, Any]:
    parsed_request = {
        "destination": state.get("destination", ""),
        "query": state.get("user_query", ""),
    }

    # Call function
    ranked_places = rank_places_by_query_rag(parsed_request)

    return {"ranked_places": ranked_places}

In [93]:
from geopy.distance import geodesic
from datetime import datetime, timedelta

# Transit Proxy Rules: To compute time and cost of transit
def compute_transit(from_coord, to_coord):
    """
    Returns estimated (time in minutes, cost in USD) based on distance.
    Cost increases with distance.
    """
    dist_km = geodesic(
        (from_coord['lat'], from_coord['lng']),
        (to_coord['lat'], to_coord['lng'])
    ).km

    if dist_km < 2:
        # Walking: free, slowest
        time = dist_km * 15   # 15 min/km
        cost = 0
    elif dist_km <= 10:
        # Metro/Bus: faster, moderate cost per km
        time = dist_km * 5    # 5 min/km
        cost = 2 * dist_km    # $2 per km
    else:
        # Taxi/Train: fastest, higher cost per km
        time = dist_km * 2    # 2 min/km
        cost = 3 * dist_km + 5   # $3 per km + base fare

    return round(time, 1), round(cost, 2)

def parse_time_str(time_str):
    """Convert HH:MM string to minutes from midnight for calculation."""
    h, m = map(int, time_str.split(":"))
    return h * 60 + m

def is_open(place, current_time_min):
    """Check if place is open at current_time_min (minutes from midnight)."""
    open_str, close_str = place["opening_hours"].split("-")
    open_min = parse_time_str(open_str)
    close_min = parse_time_str(close_str)
    return open_min <= current_time_min <= close_min

def minutes_to_hhmm(minutes: float) -> str:
    """Convert minutes from midnight to HH:MM string."""
    h = int(minutes // 60)
    m = int(minutes % 60)
    return f"{h:02d}:{m:02d}"

def get_next_open_time(place: Dict[str, Any], current_time_min: int) -> float:
    """
    Given a place with "opening_hours" like "06:00-17:00" and current_time_min,
    return the next open time (in minutes). If already open, return current_time_min.
    If it never opens later today, return None.
    """
    hours = place.get("opening_hours")
    if not hours:
        return None  # no data

    try:
        open_str, close_str = hours.split("-")
        open_min = parse_time_str(open_str)
        close_min = parse_time_str(close_str)
    except Exception:
        return None  # malformed entry

    if current_time_min < open_min:
        return open_min  # wait until opening
    elif open_min <= current_time_min < close_min:
        return current_time_min  # already open
    else:
        return None  # closed for the rest of the day


In [94]:
def build_itinerary(
    parsed_request: Dict[str, Any],
    chosen_hotel: Dict[str, Any],
    budget_remaining: float,
    ranked_places: List[Dict[str, Any]],
    daily_hours: float = 8
) -> Dict[str, Any]:
    """
    Node 5: Generate day-wise itinerary.
    """
    itinerary = []
    remaining_budget = budget_remaining
    hotel_coord = chosen_hotel["coordinates"]

    # Convert daily hours to minutes
    daily_limit_min = daily_hours * 60

    # Track places already scheduled
    scheduled_places = set()

    for day in range(1, parsed_request["days"] + 1):
        day_plan = []
        time_spent = 0
        current_coord = hotel_coord
        current_time_min = 9 * 60  # Start day at 09:00

        while time_spent < daily_limit_min:
            # Filter unscheduled places that are affordable and open
            available = [
              p for p in ranked_places
              if p["place"]["name"] not in scheduled_places
              and (p["place"]["ticket_cost"] == "Free" or p["place"]["ticket_cost"] <= remaining_budget)
              and is_open(p["place"], current_time_min)
            ]

            if not available:
            # Instead of breaking, find the next best-ranked place (even if closed now)
              candidates = [
                  p for p in ranked_places
                  if p["place"]["name"] not in scheduled_places
                  and (p["place"]["ticket_cost"] == "Free" or p["place"]["ticket_cost"] <= remaining_budget)
               ]

              if not candidates:
                  break  # Nothing left at all

              # Pick best-ranked candidate
              top_candidate = candidates[0]
              next_open_time = get_next_open_time(top_candidate["place"], current_time_min)

              if next_open_time is None:
                  # This place never opens again today → skip it
                  scheduled_places.add(top_candidate["place"]["name"])
                  continue

              # Fast-forward current time to when it opens
              wait_time = next_open_time - current_time_min
              current_time_min = next_open_time
              time_spent += wait_time

              # Now loop will retry with updated current_time_min
              continue


            # Select top interest place first
            top_place = available[0]
            place_coord = top_place["place"]["coordinates"]

            # Compute transit from current location
            transit_time, transit_cost = compute_transit(current_coord, place_coord)

            # Total time needed
            visit_duration = top_place["place"]["duration_mins"]
            total_time_needed = transit_time + visit_duration

            # Total cost needed
            ticket_cost = 0 if top_place["place"]["ticket_cost"] == "Free" else top_place["place"]["ticket_cost"]
            total_cost_needed = transit_cost + ticket_cost

            if time_spent + total_time_needed > daily_limit_min or total_cost_needed > remaining_budget:
                # Cannot fit, try next available place
                if len(available) > 1:
                    top_place = available[1]
                    place_coord = top_place["place"]["coordinates"]
                    transit_time, transit_cost = compute_transit(current_coord, place_coord)
                    visit_duration = top_place["place"]["duration_mins"]
                    total_time_needed = transit_time + visit_duration
                    ticket_cost = 0 if top_place["place"]["ticket_cost"] == "Free" else top_place["place"]["ticket_cost"]
                    total_cost_needed = transit_cost + ticket_cost
                    if time_spent + total_time_needed > daily_limit_min or total_cost_needed > remaining_budget:
                        break
                else:
                    break


            # Schedule the place
            scheduled_places.add(top_place["place"]["name"])
            day_plan.append({
                "name": top_place["place"]["name"],
                "category": top_place["place"]["category"],
                "description": top_place["place"]["description"],
                "arrival_time": minutes_to_hhmm(current_time_min + transit_time),
                "visit_duration_min": visit_duration,
                "departure_time": minutes_to_hhmm(current_time_min + transit_time + visit_duration),
                "ticket_cost": ticket_cost,
                "transit_time_min": transit_time,
                "transit_cost": transit_cost
            })

            # Update budget, time, location
            remaining_budget -= total_cost_needed
            time_spent += total_time_needed
            current_time_min += total_time_needed
            current_coord = place_coord

        itinerary.append({
            "day": day,
            "places": day_plan,
        })

    return {
        "itinerary": itinerary,
        "budget_remaining": remaining_budget
    }

In [95]:
def build_itinerary_node(state: TravelState) -> Dict[str, Any]:
    # Read the parsed request from state
    parsed_request = {
        "origin": state.get("origin", ""),
        "destination": state.get("destination", ""),
        "days": state.get("days", 0),
        "budget": state.get("budget", 0),
        "interests": state.get("interests", [])
    }
    chosen_hotel = state.get("chosen_hotel", {})
    budget_remaining = state.get("budget_remaining", 0)
    ranked_places = state.get("ranked_places", [])

    # Call function
    itinerary_data = build_itinerary(parsed_request, chosen_hotel, budget_remaining, ranked_places)

    # Return keys to merge into the state
    return itinerary_data

In [102]:
!pip install reportlab

In [103]:
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import A4
import os

In [104]:
def final_output_node(state: TravelState) -> Dict[str, Any]:
    """
    Node: Format the itinerary into a clean text output without using LLMs.
    Includes destination, budget, flight info, hotel info, daily plan, and leftover budget.
    """
    itinerary_data = state.get("itinerary", [])
    parsed_request = state.get("parsed_request", {})
    flight_info = state.get("chosen_flight", {})
    hotel_info = state.get("chosen_hotel", {})
    remaining_budget = state.get("budget_remaining", "N/A")
    destination = state.get("destination")
    budget = state.get("budget")

    # Initialize output lines

    output_lines = []

    # Destination + Budget
    output_lines.append(f"Destination: {destination}")
    output_lines.append(f"Budget: ${budget}\n")

    # Flight Info
    if flight_info:
        output_lines.append("Flight Info:")
        output_lines.append(f"  Departure: {flight_info.get('origin', 'N/A')} at {flight_info.get('departure_date', 'N/A')}")
        output_lines.append(f"  Arrival: {flight_info.get('destination', 'N/A')} at {flight_info.get('arrival_date', 'N/A')}")
        output_lines.append(f"  Airline: {flight_info.get('airline', 'N/A')}")
        output_lines.append(f"  Ticket Cost: ${flight_info.get('price', 'N/A')}\n")

    # Hotel Info
    if hotel_info:
        output_lines.append("Hotel Info:")
        output_lines.append(f"  Name: {hotel_info.get('name', 'N/A')}")
        output_lines.append(f"  Rating: {hotel_info.get('rating', 'N/A')} ")
        output_lines.append(f"  Amenities: {', '.join(hotel_info.get('amenities', []))}")
        output_lines.append(f"  Price per Night: ${hotel_info.get('price_per_night', 'N/A')}\n")

    # Daily Itinerary
    for day in itinerary_data:
        output_lines.append(f"**Day {day['day']}:**")

        if not day["places"]:
            output_lines.append("  (No places planned)\n")
            continue

        for place in day["places"]:
            name = place["name"]
            arrival = place.get("arrival_time", "N/A")
            departure = place.get("departure_time", "N/A")
            desc = place.get("description", "")
            categories = ", ".join(place.get("category", []))
            cost = place.get("ticket_cost", "N/A")

            output_lines.append(
                f"  - {arrival} → {departure}: {name} ({categories})\n"
                f"    Ticket: ${cost}\n"
                f"    {desc}\n"
            )

    # Leftover Budget
    output_lines.append(f"\n Leftover Budget: ${remaining_budget}")

    formatted = "\n".join(output_lines)
    # ---- PDF Export ----
    pdf_filename = f"itinerary_{destination}.pdf"
    doc = SimpleDocTemplate(pdf_filename, pagesize=A4)
    styles = getSampleStyleSheet()
    story = []

    for line in formatted.split("\n"):
        if line.strip().startswith("**") and line.strip().endswith("**"):  # Bold headings
            story.append(Paragraph(f"<b>{line.strip('**')}</b>", styles["Heading2"]))
        else:
            story.append(Paragraph(line, styles["Normal"]))
        story.append(Spacer(1, 6))  # small space

    doc.build(story)

    return {
        "final_output": formatted,
        "pdf_file": os.path.abspath(pdf_filename)
    }


In [105]:
# -----------------------------
# Build the graph
# -----------------------------
workflow = StateGraph(TravelState)

workflow.add_node("parse_query", parse_user_query_node)
workflow.add_node("select_flight", select_flight_node)
workflow.add_node("select_hotel", select_hotel_node)
workflow.add_node("rank_places", rank_places_node)
workflow.add_node("build_itinerary", build_itinerary_node)
workflow.add_node("final_output", final_output_node)

workflow.add_edge("parse_query", "select_flight")
workflow.add_edge("select_flight", "select_hotel")
workflow.add_edge("select_hotel", "rank_places")
workflow.add_edge("rank_places", "build_itinerary")
workflow.add_edge("build_itinerary", "final_output")

workflow.set_entry_point("parse_query")
workflow.set_finish_point("final_output")

# Compile the graph
app = workflow.compile()

# -----------------------------
# Invoke with initial state
# -----------------------------
initial_state = TravelState(
    user_query="Plan me a 3-day trip to New Delhi with $2000 budget",
    destination="",
    days=0,
    budget=0,
    interests=[],
    origin="",
    chosen_flight=None,
    chosen_hotel=None,
    ranked_places=[],
    budget_remaining=0,
    itinerary=None,
    final_output="",
    pdf_file=""
)

result = app.invoke(initial_state)

# -----------------------------
# Inspect the result
# -----------------------------
print("FINAL ITINERARY:\n", result['final_output'])
print("\n download pdf:", result['pdf_file'])

FINAL ITINERARY:
 Destination: New Delhi
Budget: $2000

Flight Info:
  Departure: San Francisco at 2025-09-15T10:00:00
  Arrival: New Delhi at 2025-09-16T23:00:00
  Airline: United Airlines
  Ticket Cost: $880

Hotel Info:
  Name: Lodhi Luxury Hotel
  Rating: 4.8 
  Amenities: Spa, Gym, Restaurant, Free Wi-Fi
  Price per Night: $200

**Day 1:**
  - 09:13 → 10:13: India Gate (Monument, landmark, culture, history)
    Ticket: $0
    A war memorial honoring soldiers, surrounded by gardens and a popular spot for evening strolls.

  - 10:28 → 12:28: National Museum, New Delhi (Museum, art, history)
    Ticket: $7
    Showcases India’s rich heritage with artifacts ranging from prehistoric times to modern era.

  - 12:39 → 14:39: Connaught Place (Shopping, Food)
    Ticket: $0
    A central business and shopping hub with colonial architecture, markets, cafes, and restaurants.

  - 14:52 → 15:22: Raj Ghat (Memorial, history)
    Ticket: $0
    The memorial site of Mahatma Gandhi, surrounded by